In [ ]:
from plastering.inferencers.inferencer import Inferencer
from plastering.feature_selector import *

from sklearn.preprocessing import LabelEncoder as LE
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import accuracy_score as ACC


class SupervisedInterface(Inferencer):

    def __init__(self,
                 target_building,
                 target_srcids,
                 ):
        super(SupervisedInterface, self).__init__(
            target_building=target_building,
            target_srcids=target_srcids
        )

        le = LE()

        self.ids, self.fd = get_data_features(target_building, None, None)

        res = get_labels(target_building)
        label = [res[srcid] for srcid in self.ids]

        try:
            self.label = le.fit_transform(label)
        except:
            pdb.set_trace()

        self.learner = RFC(n_estimators=200, criterion='entropy')
        
        
    def select_example(self):

        pass


    def update_model(self, srcid):

        self.learner.train(self.fd[srcid], self.label[srcid])


    def predict(self, target_srcids):

        return self.learner.predict(self.fd[target_srcids])


    def learn_auto(self):
        
        kf = KFold(n_splits=10, shuffle=True)
        for train, test in kf.split(X):
            self.update_model(train)
            preds = self.predict(test)
            acc = ACC(preds, self.label[test])
            print ('acc on a fold', acc)
